In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
import time

In [6]:
msize = 20000
a = numpy.random.randn(msize, msize)

In [7]:
time_start = time.time()
a = a.astype(numpy.float32)
print a.nbytes
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
#include <math.h>
  __global__ void myfunc(float *a)
  {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    a[idx] =  1 / (1+exp(-1 * a[idx]));
  }
  """)
func = mod.get_function("myfunc")
func(a_gpu, block=(1024,1,1), grid=(int(msize*msize/1024)+1,1,1))
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print time.time() - time_start
print a[0][0]
print a_doubled[0][0]

1600000000
7.12400007248
-0.974352
0.274014


In [8]:
time_start = time.time()
b = 1 / (1 + numpy.exp(-1 * a))
print time.time() - time_start
print a[0][0]
print b[0][0]

9.46700000763
-0.974352
0.274014
